# Домашнее задание 8

1. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

-> таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

-> аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

3. В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

## Задача 1

In [1]:
import pandas as pd

In [2]:
# Читаем файл с рейтингами
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# Для контроля
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
# Группируем пользователей и считаем кто сколько выставим оценок
# Сохраняем результаты в отдельный df
rating_counts = ratings.groupby('userId').count().reset_index()

# Оставляем только нужные колонки, чтобы уменьшить вычислительную нагрузку
rating_counts = rating_counts[['userId', 'rating']]
rating_counts = rating_counts [ rating_counts['rating']>100 ]

# Для контроля
rating_counts.head()

,userId,rating
3,4,204
7,8,116
14,15,1700
16,17,363
18,19,423


In [4]:
# Оставляем только нужные колонки, чтобы уменьшить вычислительную нагрузку
user_life_time = ratings[['userId', 'timestamp']]

# Групприруем данные и сразу считаем MIN и MAX
user_life_time = user_life_time.groupby('userId').agg(['min', 'max']).reset_index()

# Для контроля
user_life_time.head()

userId   timestamp            
                 min         max
0      1  1260759108  1260759205
1      2   835355395   835356246
2      3  1298861589  1298932787
3      4   949778714   949982274
4      5  1163373044  1163375145

In [5]:
# В новом столбце считаем время жизни пользователя
user_life_time['life_time'] = user_life_time['timestamp']['max'] - user_life_time['timestamp']['min']

# Оставляем только нужные колонки, чтобы убрать предупреждение дальше
# Не понял как убрать мультииндекс :(

# user_life_time['life_time'] = user_life_time['life_time']
# user_life_time['max'] = user_life_time['timestamp']['max']
# user_life_time['min'] = user_life_time['timestamp']['min']

# user_life_time = user_life_time[ ['userId', 'life_time'] ]


# Для контроля
user_life_time.head()

userId   timestamp             life_time
                 min         max          
0      1  1260759108  1260759205        97
1      2   835355395   835356246       851
2      3  1298861589  1298932787     71198
3      4   949778714   949982274    203560
4      5  1163373044  1163375145      2101

In [6]:
# Делаем left-join для таблиц `rating_counts` и `user_life_time`

In [7]:
rating_counts = rating_counts.join(user_life_time, how='left')
rating_counts.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,userId,rating,"(userId, )","(timestamp, min)","(timestamp, max)","(life_time, )"
3,4,204,4,949778714,949982274,203560
7,8,116,8,1154389340,1154474527,85187
14,15,1700,15,997937239,1469330735,471393496
16,17,363,17,1127468587,1127476640,8053
18,19,423,19,855190091,855195373,5282


## Задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [8]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [9]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [10]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [11]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [12]:
# Объединяем таблицы по правилу `outer`
table1 = rzd.merge(auto, how='outer').merge(air, how='outer').fillna(0)

# Для контроля
table1

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [13]:
# Добавляем адреса клиентов по правилу `left`
table1.merge(client_base, how='left')

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


## Задача 3

In [14]:
import datetime

In [15]:
int(datetime.datetime.strptime('01/01/2007', "%d/%m/%Y").timestamp())

1167598800

In [16]:
"""
    Тестовый df c данными о пользователе - последовательность 
    координат (широта/долгота), когда они требовались приложению для полноценной работы
"""
user_data = pd.DataFrame(
    {
        'userId': [111, 112, 113, 114, 115, 116, 117, 118, 111, 112, 113, 114, 115, 116, 117, 118],
        'latitude': [55, 47, 88, 12, 1, 123, 22, 11, 1, 3, 7, 99, 6, 8, 2, 66],
        'longitude': [1, 3, 7, 99, 6, 8, 2, 66, 55, 47, 88, 12, 1, 123, 22, 11],
        'timestamp': [1167598800, 1167598800, 1167598800, 1167598800, 1167598800, 1167598800, 1167598800, 1167598800, 1167598800, 1167598800, 1167598800, 1167598800, 1167598800, 1167598800, 1167598800, 1167598800],
    }
)
user_data

,userId,latitude,longitude,timestamp
0,111,55,1,1167598800
1,112,47,3,1167598800
2,113,88,7,1167598800
3,114,12,99,1167598800
4,115,1,6,1167598800
5,116,123,8,1167598800
6,117,22,2,1167598800
7,118,11,66,1167598800
8,111,1,55,1167598800
9,112,3,47,1167598800


In [17]:
transaction_data = pd.read_csv('transactions.csv')
transaction_data.head()

,Year,Month,ID,Product,Amount,Price,Total,Cost
0,2017,1,t001,A1,7,2904,20328,1200
1,2017,1,t002,A2,2,1896,3792,1100
2,2017,2,t003,A1,5,2904,14520,1200
3,2017,2,t004,A4,1,8618,8618,4200
4,2017,2,t005,A5,3,5175,15525,500


### Ход мыслей по задаче 3

#### Подход 1
Странно, что в данных о заказах нет ID пользователя. Так бы можно было попробовать сделать merge по user_id, но тут возникает проблема дублей данных т.к. покупок у пользователя может быть 3 штуки и координат 6 пар. Получится 18 комбинаций - не хорошо.


#### Подход 2
Я бы обогатил данные по координатам еще и данными о времени получения этих координат (лучше в формате `timestamp`).

Далее `timestamp` преобразуем в `d/m/Y` (или наоборот) и пытаемся найти максимально близкое значение из данных о покупках (хотя там тоже странно, что нет числа месяца, в котором произошла покупка). Если такое значение находится, то добавляем его в таблицу. Если по какой-то причине его нет (например, мы поставили условие, что между `timestamp` получения координа и покупкой не должно проходить более N=5 часов (секунд/минут/часов/дней/недель)), то прост ставим `0` или `NaN` или `None`.


В итоге у нас получается хоть какое-то соответствие покупок и координат.

Задачка очень похожа на вызовы Я.Такси :) и данных там должно быть больше ;)

